In [1]:
import os,torch,transformers,datasets,peft,trl

print("torch version:",torch.__version__)
print("transformers version:",transformers.__version__)
print("datasets version:",datasets.__version__)
print("peft version:",peft.__version__)
print("trl version:",trl.__version__)

torch version: 2.4.1+cu121
transformers version: 4.44.0
datasets version: 3.0.1
peft version: 0.4.0
trl version: 0.11.2


In [4]:
import os
import torch
import torch.nn as nn
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from accelerate import Accelerator

In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "../../../autodl-tmp/ZhipuAI/glm-4-9b-chat"

# The instruction dataset to use
dataset_name = "../../data/2025-dataset.json"

# Fine-tuned model name
new_model = "glm-4-9b-chat-diabetes-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.6

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training 原来是4
per_device_train_batch_size = 1

# Batch size per GPU for evaluation 原来是4
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for 原来是1
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer) 原来是2e-4
learning_rate = 3e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 10

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [6]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = "left" # Fix weird overflow issue with fp16 training


# Load dataset (you can process it here)
dataset = load_dataset("json",data_files=dataset_name,split="train")

def format_chat_template(row):
    row_json = [{"role": row["messages"][0]["role"], "content": row["messages"][0]["content"]},
                {"role": row["messages"][1]["role"], "content": row["messages"][1]["content"]},
                {"role": row["messages"][2]["role"], "content": row["messages"][2]["content"]},
               ]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc= 4,
)



In [7]:
dataset.remove_columns(["messages"])

Dataset({
    features: ['text'],
    num_rows: 2270
})

In [8]:
# Split the dataset into a training and validation set
# You can specify the ratio or the number of validation examples
dataset = dataset.train_test_split(test_size=0.2)  # For example, 20% for validation

train_dataset = dataset['train']
val_dataset = dataset['test']

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [ ]:
print(model.state_dict().keys())

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Modify the trainer initialization to include the train_dataset and eval_dataset
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,  # Use the split training dataset
    eval_dataset=val_dataset,  # Use the split validation dataset
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

In [6]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [7]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True, 
# bf16
    torch_dtype=torch.float16,
    device_map=device_map,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [8]:
save_directory = "../../../autodl-tmp/fine_tuned_chatglm_for_diabetes"

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('../autodl-tmp/fine_tuned_chatglm_for_diabetes/tokenizer_config.json',
 '../autodl-tmp/fine_tuned_chatglm_for_diabetes/special_tokens_map.json',
 '../autodl-tmp/fine_tuned_chatglm_for_diabetes/tokenizer.model',
 '../autodl-tmp/fine_tuned_chatglm_for_diabetes/added_tokens.json')

In [10]:
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

0

In [11]:
del model

NameError: name 'model' is not defined

In [2]:
from transformers import AutoModel, AutoTokenizer

load_directory="../../../autodl-tmp/fine_tuned_chatglm_for_diabetes"

model = AutoModel.from_pretrained(load_directory,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(load_directory,trust_remote_code=True)
tokenizer.padding_side="left"
#tokenizer.pad_token="[PAD]"
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to DataParallel for multi-GPU usage
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

model = model.to(device)
model.eval()  # Ensure evaluation mode



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(151552, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-39): 40 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): SdpaAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in

In [6]:
messages = [{
                "role": "system",
                "content": "你是一个专业的医生，请基于诊疗指南，为以下患者提供综合的管理意见:"
            },
            {
                "role": "user",
                "content":"我是一位30岁的男性，头有点晕，检测空腹血糖为10.0mmol/L，餐后为15.0mol/L，BMI 30,我该如何调整我的饮食来控制血糖？"
            }
           ]


prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding= True, truncation=True).to(device)

outputs = model.generate(**inputs, max_length=2000, num_return_sequences=1)

#outputs = model.module.generate(**inputs, max_length=1000, num_return_sequences=1)

# temperature=0.1,top_p = 0.95,top_k= 5) if isinstance(model, nn.DataParallel) else model.generate(**inputs, max_length=1000, num_return_sequences=1,temperature=0.1,top_p = 0.95,top_k= 5)

text = tokenizer.decode(outputs[0], skip_special_tokens=False)

print("Output:")
print(text.split("<|assistant|>")[1].split("<|user|>")[0])

Output:
 
根据您提供的信息，以下是针对您的健康建议:您的血糖水平高于正常值，提示可能为糖尿病。近期应检测空腹血糖和餐后2小时血糖，并检测糖化血红蛋白，以明确糖尿病的确诊，同时注意血压和血脂的检测。在糖尿病药物治疗方面，建议您咨询主管医生，在医生指导下使用。以下是为您提供的饮食建议，请注意适量运动，并定期检测血糖：建议您每日总卡路里摄入在2500左右，其中碳水化合物占总摄入量的50%到60%，蛋白质占15%到20%，脂肪占20%到25%。具体安排建议为：早餐：燕麦片50克，牛奶250毫升，煮鸡蛋1个，或煮鸡蛋1个，豆腐100克，番茄1个；午餐：米饭150克，清蒸鱼150克，拌黄瓜100克；晚餐：米饭150克，瘦肉150克，青菜250克。在运动方面，建议您每日进行30分钟的有氧运动，如散步、慢跑等。同时，注意每日检测血糖，并及时联系专科医生复诊。您的血糖控制略高，建议检测糖化血红蛋白，了解近3个月的血糖水平，并联系专科医生复诊。您目前血压和血脂水平较高，建议尽快检测，如异常，尽快联系专科医生复诊。建议您保持每日至少6小时以上的低强度运动（如快步走）。注意饮食，以低糖，低脂，高蛋白为主。保持每日至少1次的水温适宜的淋浴。保持每日心情舒畅。祝您健康！ 
